Codigo do tcc

In [1]:
!pip3 install transformers
!pip3 install dl-translate
!pip3 install emoji
!pip3 install tensorflow
!pip3 install numpy

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import dl_translate as dlt

2023-08-12 11:40:47.486849: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-12 11:40:47.522852: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-12 11:40:47.523816: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 11:40:48.350241: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/lincoln81/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mt = dlt.TranslationModel()

In [17]:
def translater_frases(frase):
  text_hi = frase
  translater = mt.translate(text_hi, source=dlt.lang.PORTUGUESE, target=dlt.lang.ENGLISH)
  return translater

def classification_frases(frase):
  input = tokenizerR.encode(frase, return_tensors="tf")
  output = modelR.predict(input)[0]
  prediction = tf.nn.softmax(output, axis=1).numpy()
  sentiment = np.argmax(prediction) 

  return sentiment

## Modelo de classificação de Tweets, treinado com 23 milhões de exemplos em inglês, sobre covid.

In [5]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizerR = AutoTokenizer.from_pretrained("rabindralamsal/BERTsent")

modelR = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/BERTsent")


2023-08-12 11:41:08.839225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-12 11:41:08.839479: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/BERTsent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions 

In [9]:
example_tweet = "The NEET exams show our Govt in a poor light: unresponsiveness to genuine concerns; admit cards not delivered to aspirants in time; failure to provide centres in towns they reside, thus requiring unnecessary & risky travels. What a disgrace to treat our #Covid warriors like this!"
    
input = tokenizerR.encode(example_tweet, return_tensors="tf")
output = modelR.predict(input)[0]
prediction = tf.nn.softmax(output, axis=1).numpy()
sentiment = np.argmax(prediction)
    
print(prediction)
print(sentiment)

1/1 [==============================] - 0s 138ms/step
[[0.97267216 0.02368475 0.00364307]]
0


In [10]:
dataset = pd.read_csv("./TweetSentBR.csv")

In [11]:
dataset.drop(columns = ['id', 'id_twitter'],inplace=True)

In [12]:
dataset

,text,sentiment
0,Que coisa linda! O Programa #encontro estava m...,1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,quem viu aquela lutadora modela barbuda tatuad...,0
4,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...
11560,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11561,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11562,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11563,"Que orgulho de ti, @sportrecife! #Encontro",1


In [13]:
positive = dataset[dataset['sentiment'] == '1'].head(10)

In [14]:
positive['textEnglish'] = positive['text'].apply(translater_frases)

In [25]:
positive['sentimentEnglish'] = positive['textEnglish'].apply(classification_frases)

1/1 [==============================] - 0s 59ms/step


In [33]:
positive['sentimentEnglish'] = positive['sentimentEnglish'].replace(2, '1').replace(0, '-1').replace(1, '0')

In [46]:
positive.to_csv('positive.csv', index=False)

In [36]:
negative = dataset[dataset['sentiment'] == '-1'].head(10)

In [38]:
negative['textEnglish'] = negative['text'].apply(translater_frases)

In [43]:
negative['sentimentEnglish'] = negative['textEnglish'].apply(classification_frases)

1/1 [==============================] - 0s 113ms/step


In [44]:
negative['sentimentEnglish'] = negative['sentimentEnglish'].replace(2, '1').replace(0, '-1').replace(1, '0')

In [47]:
negative.to_csv('negative.csv', index=False)